In [ ]:
import numpy as np
from keras.models import model_from_json
import operator
import cv2
import time
import sys, os
from gtts import gTTS

# Loading the model
json_file = open("ourmodel-bw.json", "r")
model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(model_json)
# load weights into new model
loaded_model.load_weights("ourmodel-bw.h5")
print("Loaded model from disk")

cap = cv2.VideoCapture(0)

# Category dictionary
categories = {0: 'Nothing', 1: 'Hello', 2: 'Yes', 3: 'No', 4: 'ThankYou', 5: 'Help', 6: 'You', 7: 'Wow', 8: 'What',
             9: 'Emergency', 10: 'Done',11: 'Climb',12: 'PLay',13: 'Liedown',14: 'More',15: 'Cold',16: 'Iloveyou',17: 'Peace',18: 'Dislike',19: 'Coffee'
             }
while True:
    _, frame = cap.read()
    # Simulating mirror image
    frame = cv2.flip(frame, 1)
    
    # Got this from collect-data.py
    # Coordinates of the ROI
    x1 = int(0.5*frame.shape[1])
    y1 = 10
    x2 = frame.shape[1]-10
    y2 = int(0.5*frame.shape[1])
    # Drawing the ROI
    # The increment/decrement by 1 is to compensate for the bounding box
    cv2.rectangle(frame, (x1-1, y1-1), (x2+1, y2+1), (0,255,0) ,1)
    # Extracting the ROI
    roi = frame[y1:y2, x1:x2]
    
    # Resizing the ROI so it can be fed to the model for prediction
    roi = cv2.resize(roi, (64, 64)) 
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    _, test_image = cv2.threshold(roi, 120, 255, cv2.THRESH_BINARY)
    cv2.imshow("test", test_image)
    # Batch of 1
    result = loaded_model.predict(test_image.reshape(1, 64, 64, 1))
    prediction = {'Nothing' : result[0][12],
                  'Hello': result[0][6], 
                  'Yes': result[0][18], 
                  'No': result[0][11],
                  'ThankYou': result[0][15],
                  'Help': result[0][7],
                  'You': result[0][19],
                  'Wow': result[0][17],
                  'What': result[0][16],
                  'Emergency': result[0][5],
                  'Done': result[0][4],
                  'Climb': result[0][0],
                  'Play': result[0][14],
                  'Liedown': result[0][9],
                  'More': result[0][10],
                  'Cold': result[0][2],
                  'Iloveyou': result[0][8],
                  'Peace': result[0][13],
                  'Dislike': result[0][3],
                  'Coffee': result[0][1]
                  }
    # Sorting based on top prediction
    prediction = sorted(prediction.items(), key=operator.itemgetter(1), reverse=True)
    
    # Displaying the predictions
    cv2.putText(frame, prediction[0][0], (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 3, (0,0,255), 3)    
    cv2.imshow("Frame", frame)
    k = [prediction[0][0]]
    interrupt = cv2.waitKey(10)
    if interrupt & 0xFF == 27: # esc key
        break
    
         
cap.release()
cv2.destroyAllWindows()
print(k)
prediction_text =str(k)
speech_object = gTTS(text= prediction_text, lang= 'en', slow= False)
speech_object.save("myspeak.mp3")
os.system("start myspeak.mp3")
    
   

Loaded model from disk
